In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import pkg_resources
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gymnasium as gym
import stable_baselines3 as sb3
import stable_baselines3.common.logger as logger
import stable_baselines3.common.callbacks as callbacks
import stable_baselines3.common.env_checker as env_checker
from dm_control import mjcf
from dm_control.rl.control import PhysicsError
import imageio
import scipy.spatial
import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric as pyg
import torch.nn.functional as F
import torch_geometric.nn as gnn
import torch_geometric.loader as pyg_loader
import pytorch_lightning as pl
import torchmetrics
from torch.utils.data import Dataset
from pathlib import Path
from typing import Tuple, Callable, Optional, List, Union
from tqdm import trange
from dm_control.rl.control import PhysicsError
from PIL import Image

from flygym.arena.mujoco_arena import FlatTerrain
from flygym.envs.nmf_mujoco import NeuroMechFlyMuJoCo, MuJoCoParameters
from flygym.state import stretched_pose
import flygym.util.vision as vision
import flygym.util.config as config
from flygym.arena import BaseArena
from flygym.arena.mujoco_arena import OdorArena, FlatTerrain, GappedTerrain, BlocksTerrain
from flygym.util.data import color_cycle_rgb

from rl_navigation import ObstacleOdorArena, NMFNavigation

pygame 2.5.1 (SDL 2.28.2, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/lightning_fabric/__init__.py:36: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning_fabric')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__("pkg_resources").declare_namespace(__name__)
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site

Sanity check on MDP task

In [3]:
def make_arena():
    terrain_arena = FlatTerrain(ground_alpha=1)
    odor_arena = ObstacleOdorArena(
        terrain=terrain_arena,
        obstacle_positions=np.array([(7.5, 0)]),
        obstacle_radius=1,
        odor_source=np.array([[15, 0, 2]]),
        marker_size=0.5,
        obstacle_colors=(0, 0, 0, 1),
    )
    return odor_arena

In [4]:
sim = NMFNavigation(
    arena_factory=make_arena,
    test_mode=True,
    debug_mode=True,
    decision_dt=0.1,
)
# env_checker.check_env(sim)

In [5]:
# sim.reset()
for i in trange(2):
    obs, reward, terminated, truncated, info = sim.step(np.array([0]))
    if terminated:
        print("Terminated")
        break
sim.controller.save_video("test_3a.mp4", stabilization_time=0)

  0%|          | 0/2 [00:00<?, ?it/s]/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
 50%|█████     | 1/2 [00:04<00:04,  4.33s/it]

fly_pos: [-0.16976582 -0.06271822], final reward=-1.7676489587760318, state=seeking
  dist rew=-0.169895, danger=0.799, action diff=0.000, tgt rew=0.000
  dist=15.170


100%|██████████| 2/2 [00:07<00:00,  3.71s/it]

fly_pos: [0.47388283 0.23758284], final reward=-1.0400181903458359, state=seeking
  dist rew=0.641836, danger=0.841, action diff=0.000, tgt rew=0.000
  dist=14.528


In [6]:
sim.reset(spawn_pos=[0, 5, 0.2])
for i in trange(2):
    obs, reward, terminated, truncated, info = sim.step(np.array([0]))
    if terminated:
        print("Terminated")
        break
sim.controller.save_video("test_3b.mp4", stabilization_time=0)

resetting environment


 50%|█████     | 1/2 [00:02<00:02,  2.87s/it]

fly_pos: [0.0246491 5.1837409], final reward=-1.0329213942156095, state=seeking
  dist rew=-0.847154, danger=0.093, action diff=0.000, tgt rew=0.000
  dist=15.847


100%|██████████| 2/2 [00:06<00:00,  3.26s/it]

fly_pos: [0.59004269 4.85472576], final reward=-0.10560701370095127, state=seeking
  dist rew=0.641391, danger=0.373, action diff=0.000, tgt rew=0.000
  dist=15.206


In [7]:
sim.reset()
for i in trange(15):
    obs, reward, terminated, truncated, info = sim.step(np.array([0.5 if i < 5 else 0]))
    if terminated:
        print("Terminated")
        break
sim.controller.save_video("test_3c.mp4", stabilization_time=0)

resetting environment


  7%|▋         | 1/15 [00:03<00:42,  3.03s/it]

fly_pos: [0.12608748 0.84223408], final reward=-2.1388919571329446, state=seeking
  dist rew=0.102261, danger=0.996, action diff=0.250, tgt rew=0.000
  dist=14.898


 13%|█▎        | 2/15 [00:06<00:39,  3.05s/it]

fly_pos: [0.53565776 0.76884876], final reward=-1.0023499305836974, state=seeking
  dist rew=0.412977, danger=0.708, action diff=0.000, tgt rew=0.000
  dist=14.485


...

fly_pos: [ 4.27111439 -9.70476212], final reward=-0.37211699700450396, state=seeking
  dist rew=-0.372117, danger=0.000, action diff=0.000, tgt rew=0.000
  dist=14.467


 93%|█████████▎| 14/15 [00:34<00:02,  2.30s/it]

fly_pos: [  4.6194063  -11.02754679], final reward=-0.6778451526022913, state=seeking
  dist rew=-0.677845, danger=0.000, action diff=0.000, tgt rew=0.000
  dist=15.145


100%|██████████| 15/15 [00:36<00:00,  2.43s/it]

fly_pos: [  4.71283152 -12.25050455], final reward=-0.8521444660176503, state=seeking
  dist rew=-0.852144, danger=0.000, action diff=0.000, tgt rew=0.000
  dist=15.997


In [8]:
def make_env():
    terrain_arena = FlatTerrain(ground_alpha=1)
    arena = ObstacleOdorArena(
        terrain=terrain_arena,
        obstacle_positions=np.array([(7.5, 0)]),
        obstacle_radius=1,
        odor_source=np.array([[15, 0, 2]]),
        marker_size=0.5,
        obstacle_colors=(0, 0, 0, 1),
    )
    sim = NMFNavigation(
        arena_factory=make_arena,
        test_mode=False,
        debug_mode=False,
    )
    return sim

from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env

num_procs = 12
# vec_env = SubprocVecEnv([make_env for i in range(num_procs)])
vec_env = make_vec_env(make_env, n_envs=num_procs, vec_env_cls=SubprocVecEnv)

/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing

In [9]:
vec_env.action_space

Box(-1.0, 1.0, (1,), float32)

In [10]:

# terrain_arena = FlatTerrain(ground_alpha=1)
# arena = ObstacleOdorArena(
#     terrain=terrain_arena,
#     obstacle_positions=np.array([(7.5, 1.5), (12.5, -1.5)]),
#     odor_source=np.array([[20, 0, 2]]),
#     marker_size=0.5,
#     obstacle_colors=(0.14, 0.14, 0.2, 1),
# )
# sim = NMFNavigation(
#     arena=arena,
#     test_mode=False,
# )
# env_checker.check_env(sim)

np.random.seed(0)
sb3.common.utils.set_random_seed(0, using_cuda=True)

start_from = None
train = True

log_dir = "logs/trial_23b"
checkpoint_callback = callbacks.CheckpointCallback(
    save_freq=100,
    save_path=log_dir,
    name_prefix="trial_23b",
    save_replay_buffer=True,
    save_vecnormalize=True,
    verbose=2,
)
my_logger = logger.configure(log_dir, ["tensorboard", "stdout", "csv"])
if start_from is None:
    model = sb3.SAC(
        "MlpPolicy",
        # env=sim,
        env=vec_env,
        policy_kwargs={"net_arch": [32, 32]},
        verbose=2,
        learning_rate=0.01,
    )
else:
    model = sb3.SAC.load(start_from)
    model.set_env(vec_env)
    print(model.verbose, model.learning_rate, model.policy_kwargs)
model.set_logger(my_logger)

if train:
    model.learn(total_timesteps=200_000, progress_bar=True, callback=checkpoint_callback)
    model.save("models/trial_23b")

Logging to logs/trial_23b
Using cuda device


Output()

/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/s

/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/s

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 15.5     |
|    ep_rew_mean     | -15.7    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 2        |
|    time_elapsed    | 89       |
|    total_timesteps | 252      |
| train/             |          |
|    actor_loss      | -0.373   |
|    critic_loss     | 0.784    |
|    ent_coef        | 0.896    |
|    ent_coef_loss   | -0.185   |
|    learning_rate   | 0.01     |
|    n_updates       | 12       |
---------------------------------


...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 40.8     |
|    ep_rew_mean     | 5.94     |
| time/              |          |
|    episodes        | 5468     |
|    fps             | 3        |
|    time_elapsed    | 62343    |
|    total_timesteps | 199284   |
| train/             |          |
|    actor_loss      | -0.825   |
|    critic_loss     | 6.58     |
|    ent_coef        | 0.0361   |
|    ent_coef_loss   | -1.2     |
|    learning_rate   | 0.01     |
|    n_updates       | 16598    |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 40.5     |
|    ep_rew_mean     | 6.39     |
| time/              |          |
|    episodes        | 5472     |
|    fps             | 3        |
|    time_elapsed    | 62392    |
|    total_timesteps | 199440   |
| train/             |          |
|    actor_loss      | 0.782    |
|    critic_loss     | 6.63     |
|    ent_coef        | 0.0309   |
|    ent_coef_loss   | -0.716   |
|    learning_rate   | 0.01     |
|    n_updates       | 16611    |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 40.8     |
|    ep_rew_mean     | 6.27     |
| time/              |          |
|    episodes        | 5476     |
|    fps             | 3        |
|    time_elapsed    | 62421    |
|    total_timesteps | 199524   |
| train/             |          |
|    actor_loss      | 0.353    |
|    critic_loss     | 7.18     |
|    ent_coef        | 0.028    |
|    ent_coef_loss   | -0.925   |
|    learning_rate   | 0.01     |
|    n_updates       | 16618    |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 41.3     |
|    ep_rew_mean     | 6.72     |
| time/              |          |
|    episodes        | 5480     |
|    fps             | 3        |
|    time_elapsed    | 62460    |
|    total_timesteps | 199644   |
| train/             |          |
|    actor_loss      | -0.241   |
|    critic_loss     | 8.81     |
|    ent_coef        | 0.025    |
|    ent_coef_loss   | 0.67     |
|    learning_rate   | 0.01     |
|    n_updates       | 16628    |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 40.5     |
|    ep_rew_mean     | 6.81     |
| time/              |          |
|    episodes        | 5484     |
|    fps             | 3        |
|    time_elapsed    | 62516    |
|    total_timesteps | 199824   |
| train/             |          |
|    actor_loss      | -0.578   |
|    critic_loss     | 6.49     |
|    ent_coef        | 0.0263   |
|    ent_coef_loss   | 0.541    |
|    learning_rate   | 0.01     |
|    n_updates       | 16643    |
---------------------------------
